# 1. Read all csv files

In [ ]:
import pandas as pd


providers_df = pd.read_csv(r'C:\Users\JAI\providers_data.csv')
receivers_df = pd.read_csv(r'C:\Users\JAI\receivers_data.csv')
food_listings_df = pd.read_csv(r'C:\Users\JAI\food_listings_data.csv')


print("Providers:")
print(providers_df.head())

print("\nReceivers:")
print(receivers_df.head())

print("\nFood Listings:")
print(food_listings_df.head())




In [ ]:
claims_df = pd.read_csv(r'C:\Users\JAI\claims_data (1).csv')
print("\nClaims(1):")
print(claims_df.head())

In [ ]:
import pandas as pd

#  2. Load csv files

In [ ]:
providers = pd.read_csv("providers_data.csv")
receivers = pd.read_csv("receivers_data.csv")
food_listings = pd.read_csv("food_listings_data.csv")
claims = pd.read_csv("claims_data (1).csv")


print(providers.head())
print(receivers.head())
print(food_listings.head())
print(claims.head())

# 3. Clean providers.csv

In [ ]:
providers = providers.loc[:, ~providers.columns.str.contains('^Unnamed')]
providers.drop_duplicates(inplace=True)
providers.dropna(inplace=True)

print(providers)

In [ ]:
providers['Name'] = providers['Name'].str.strip().str.title()
providers['Type'] = providers['Type'].str.strip().str.title()
providers['City'] = providers['City'].str.strip().str.title()
providers['Address'] = providers['Address'].str.strip()
providers['Contact'] = providers['Contact'].astype(str).str.strip()

In [ ]:
print("cleaned providers data:")
print(providers.head())

# 4. Clean receivers.csv

In [ ]:
receivers = receivers.loc[:, ~receivers.columns.str.contains('^Unnamed')]
receivers.drop_duplicates(inplace=True)
receivers.dropna(inplace=True)

print(receivers)

In [ ]:

receivers['name'] = receivers['name'].str.strip().str.title()
receivers['type'] = receivers['type'].str.strip().str.title()
receivers['city'] = receivers['city'].str.strip().str.title()
receivers['contact'] = receivers['contact'].astype(str).str.strip()


In [ ]:
print("cleaned receivers data:")
print(receivers.head())

# 5. Clean food_listings csv files

In [ ]:
food_listings = food_listings.loc[:, ~food_listings.columns.str.contains('^Unnamed')]
food_listings.drop_duplicates(inplace=True)
food_listings.dropna(subset=['Food_Name', 'Quantity', 'Expiry_Date'], inplace=True)

print(food_listings)

In [ ]:
food_listings['Food_Name'] = food_listings['Food_Name'].str.strip().str.title()
food_listings['Provider_Type'] = food_listings['Provider_Type'].str.strip().str.title()
food_listings['Location'] = food_listings['Location'].str.strip().str.title()
food_listings['Food_Type'] = food_listings['Food_Type'].str.strip().str.title()
food_listings['Meal_Type'] = food_listings['Meal_Type'].str.strip().str.title()


# convert types

In [ ]:
food_listings['Quantity'] = pd.to_numeric(food_listings['Quantity'], errors='coerce')
food_listings['Expiry_Date'] = pd.to_datetime(food_listings['Expiry_Date'], errors='coerce')

food_listings.dropna(subset=['Quantity', 'Expiry_Date'], inplace=True)

print("cleaned food_listings data:")
print(food_listings.head())

# 6. Clean claims.csv

In [ ]:
claims = claims.loc[:, ~claims.columns.str.contains('^Unnamed')]
claims.drop_duplicates(inplace=True)
claims.dropna(subset=['Food_ID', 'Receiver_ID', 'Status', 'Timestamp'], inplace=True)

print(claims)

In [ ]:
claims['Status'] = claims['Status'].str.strip().str.title()
claims['Timestamp'] = pd.to_datetime(claims['Timestamp'], errors='coerce')
claims.dropna(subset=['Timestamp'], inplace=True)

print("claeaned claims data:")
print(claims.head())


# 7. Save cleaned files

In [ ]:
providers.to_csv("cleaned_providers.csv", index=False)
receivers.to_csv("cleaned_receivers.csv", index=False)
food_listings.to_csv("cleaned_food_listings.csv", index=False)
claims.to_csv("cleaned_claims.csv", index=False)

print("Cleaned Providers:")
print(providers.head())

print("\nCleaned Receivers:")
print(receivers.head())

print("\nCleaned Food Listings:")
print(food_listings.head())

print("\nCleaned Claims:")
print(claims.head())


Insert CSV into PostgreSQL tables using python

In [ ]:
#pip install psycopg2

In [ ]:
# Python code to Insert CSV data

In [ ]:
import pandas as pd
import psycopg2


In [ ]:
#connection details

In [ ]:
conn = psycopg2.connect(
    host = "localhost",
    database= "food_wastage_db",
    user="postgres",
    password="Kirthu7625",
    port="5432"
)
cursor = conn.cursor()
print("Connected to PostgreSQL database successfully!")

In [ ]:
#Load cleaned CSVs

In [ ]:
providers = pd.read_csv("cleaned_providers.csv")
receivers = pd.read_csv("cleaned_receivers.csv")
food_listings = pd.read_csv("cleaned_food_listings.csv")
claims = pd.read_csv("cleaned_claims.csv")


In [ ]:
# Insert into tables

In [ ]:
def insert_data(df, insert_query):
    for _, row in df.iterrows():
        cursor.execute(insert_query, tuple(row))

insert_data(providers,"""
   INSERT INTO providers(provider_id, name, type, address, city, contact)
   VALUES (%s, %s, %s, %s, %s, %s)
   ON CONFLICT (provider_id) DO NOTHING
""")

insert_data(receivers, """
     INSERT INTO receivers (receiver_id, name, type, city, contact)
     VALUES (%s, %s, %s, %s, %s)
     ON CONFLICT (receiver_id) DO NOTHING
""")

insert_data(food_listings, """
      INSERT INTO food_listings(food_id, food_name, quantity, expiry_date,
       provider_id, provider_type, location, food_type, meal_type)
       VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
       ON CONFLICT (food_id) DO NOTHING
""")

insert_data(claims, """
      INSERT INTO claims (claim_id, food_id, receiver_id, status, timestamp)
      VALUES (%s, %s, %s, %s, %s)
      ON CONFLICT (claim_id) DO NOTHING
""")


conn.commit()
cursor.close()
conn.close()

print("Data inserted PostgreSQL tables successfully!")

In [5]:
import psycopg2
import pandas as pd

def print_table_data():
    try:
        # Connect to PostgreSQL
        conn = psycopg2.connect(
            dbname="food_wastage_db",
            user="postgres",
            password="Kirthu7625",  # ← Replace with your actual password
            host="localhost",
            port="5432"
        )
        cursor = conn.cursor()

        # List of tables
        tables = ["providers", "receivers", "food_listings", "claims"]

        for table in tables:
            print(f"\n🔹 {table.upper()} TABLE:")
            try:
                df = pd.read_sql(f"SELECT * FROM {table} LIMIT 5;", conn)
                print(df)
            except Exception as e:
                print(f"Error reading {table}: {e}")

        cursor.close()
        conn.close()

    except Exception as e:
        print("❌ Could not connect to the database.")
        print(e)

# Call the function
print_table_data()


🔹 PROVIDERS TABLE:


C:\Users\JAI\AppData\Local\Temp\ipykernel_10172\2242608032.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table} LIMIT 5;", conn)


   provider_id                         name           type  \
0            1             Gonzales-Cochran    Supermarket   
1            2  Nielsen, Johnson And Fuller  Grocery Store   
2            3                 Miller-Black    Supermarket   
3            4   Clark, Prince And Williams  Grocery Store   
4            5               Coleman-Farley  Grocery Store   

                                             address            city  \
0  74347 Christopher Extensions\nAndreamouth, OK ...     New Jessica   
1           91228 Hanson Stream\nWelchtown, OR 27136     East Sheena   
2  561 Martinez Point Suite 507\nGuzmanchester, W...  Lake Jesusview   
3     467 Bell Trail Suite 409\nPort Jesus, IA 61188     Mendezmouth   
4  078 Matthew Creek Apt. 319\nSaraborough, MA 53978   Valentineside   

                contact  
0       +1-600-220-0480  
1  +1-925-283-8901x6297  
2      001-517-295-2206  
3      556.944.8935x401  
4          193.714.6577  

🔹 RECEIVERS TABLE:
   receiver_id    

C:\Users\JAI\AppData\Local\Temp\ipykernel_10172\2242608032.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table} LIMIT 5;", conn)
C:\Users\JAI\AppData\Local\Temp\ipykernel_10172\2242608032.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table} LIMIT 5;", conn)
C:\Users\JAI\AppData\Local\Temp\ipykernel_10172\2242608032.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {table} LIMIT 5;", conn)


   claim_id  food_id  receiver_id     status           timestamp
0         1      164          908    Pending 2025-03-05 05:26:00
1         2      353          391  Cancelled 2025-03-11 10:24:00
2         3      626          492  Completed 2025-03-21 00:59:00
3         4       61          933  Cancelled 2025-03-04 09:08:00
4         5      345          229    Pending 2025-03-14 15:17:00


Running SQL Queries

Now we will execute seventeen SQL queries to analyze food wastage management.
    

1.Query to Get the all providers


In [1]:
import psycopg2
import pandas as pd

# PostgreSQL connection function
def get_connection():
    return psycopg2.connect(
        dbname="food_wastage_db",
        user="postgres",
        password="Kirthu7625",  # change to your actual password
        host="localhost",
        port="5432"
    )

In [2]:
query = "SELECT * FROM providers;"
df = pd.read_sql(query, get_connection())
print("1. Show all providers")
df

C:\Users\JAI\AppData\Local\Temp\ipykernel_888\2696472897.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, get_connection())


1. Show all providers


,provider_id,name,type,address,city,contact
0,1,Gonzales-Cochran,Supermarket,"74347 Christopher Extensions\nAndreamouth, OK ...",New Jessica,+1-600-220-0480
1,2,"Nielsen, Johnson And Fuller",Grocery Store,"91228 Hanson Stream\nWelchtown, OR 27136",East Sheena,+1-925-283-8901x6297
2,3,Miller-Black,Supermarket,"561 Martinez Point Suite 507\nGuzmanchester, W...",Lake Jesusview,001-517-295-2206
3,4,"Clark, Prince And Williams",Grocery Store,"467 Bell Trail Suite 409\nPort Jesus, IA 61188",Mendezmouth,556.944.8935x401
4,5,Coleman-Farley,Grocery Store,"078 Matthew Creek Apt. 319\nSaraborough, MA 53978",Valentineside,193.714.6577
...,...,...,...,...,...,...
995,996,"Vasquez, Ruiz And Flowers",Restaurant,"84308 Justin Stravenue\nNew Amberside, NE 53447",Williamview,+1-319-378-7627x0682
996,997,Garza-Williams,Catering Service,"08864 Figueroa Radial Suite 948\nJennaberg, AZ...",East Rossside,001-924-441-3963x746
997,998,Novak Group,Grocery Store,"934 Zachary Run\nMelissamouth, WY 02729",Joshuastad,(903)642-1969x3300
998,999,Moody Ltd,Grocery Store,"17580 Ernest Hills\nLake Michaelmouth, OR 56416",Stevenchester,637.300.3664x4880


2.Query to find Recent 10 claims (most recent first)

In [3]:
query = """
SELECT * FROM claims
ORDER BY timestamp DESC
LIMIT 10;
"""
df = pd.read_sql(query, get_connection())
print("2. Recent 10 claims")
df

C:\Users\JAI\AppData\Local\Temp\ipykernel_888\3867817512.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, get_connection())


2. Recent 10 claims


,claim_id,food_id,receiver_id,status,timestamp
0,804,614,655,Cancelled,2025-03-21 06:12:00
1,423,915,344,Pending,2025-03-21 05:41:00
2,535,663,329,Completed,2025-03-21 05:22:00
3,479,216,923,Pending,2025-03-21 05:08:00
4,471,11,337,Cancelled,2025-03-21 04:51:00
5,26,959,233,Cancelled,2025-03-21 04:34:00
6,466,699,285,Pending,2025-03-21 04:34:00
7,338,609,301,Pending,2025-03-21 04:08:00
8,185,486,585,Pending,2025-03-21 04:05:00
9,601,964,458,Completed,2025-03-21 03:54:00


3.Query to find Claims per day(last 30 entries)

In [4]:
query = """
SELECT DATE(timestamp) AS claim_date, COUNT(*) AS total_claims
FROM claims
GROUP BY DATE(timestamp)
ORDER BY claim_date DESC
LIMIT 30;
"""
df = pd.read_sql(query, get_connection())
print("3. Claims per day")
df

3. Claims per day


C:\Users\JAI\AppData\Local\Temp\ipykernel_888\1961965634.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, get_connection())


,claim_date,total_claims
0,2025-03-21,19
1,2025-03-20,45
2,2025-03-19,50
3,2025-03-18,57
4,2025-03-17,54
5,2025-03-16,45
6,2025-03-15,51
7,2025-03-14,51
8,2025-03-13,62
9,2025-03-12,52


4.Query to find Count of food listings by type

In [5]:
query = "SELECT food_type, COUNT(*) AS total_items FROM food_listings GROUP BY food_type;"
df = pd.read_sql(query, get_connection())
print("4. Count of food listings by type")
df

4. Count of food listings by type


C:\Users\JAI\AppData\Local\Temp\ipykernel_888\2994242998.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, get_connection())


,food_type,total_items
0,Non-Vegetarian,330
1,Vegetarian,336
2,Vegan,334


5.Query to find total number of Receivers

In [6]:
query = "SELECT COUNT(*) FROM receivers;"
df = pd.read_sql(query, get_connection())
print("5. Total number of receivers")
df


5. Total number of receivers


C:\Users\JAI\AppData\Local\Temp\ipykernel_888\3356762491.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, get_connection())


,count
0,1000


6.Query to get List food claimed status

In [7]:
query = "SELECT * FROM claims WHERE status = 'Claimed';"
df = pd.read_sql(query, get_connection())
print("6. Claimed food")
df

6. Claimed food


C:\Users\JAI\AppData\Local\Temp\ipykernel_888\713366520.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, get_connection())


,claim_id,food_id,receiver_id,status,timestamp


7.Query  to get Foods listed in current month.

In [8]:
query = """
SELECT * FROM food_listings
WHERE DATE_TRUNC('month', listed_date) = DATE_TRUNC('month', CURRENT_DATE);
"""
df = pd.read_sql(query, get_connection())
print("7. Foods listed in current month")
df

7. Foods listed in current month


C:\Users\JAI\AppData\Local\Temp\ipykernel_888\2912398852.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, get_connection())


,food_id,food_name,quantity,expiry_date,provider_id,provider_type,location,food_type,meal_type,listed_date
0,1,Bread,43,2025-03-17,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast,2025-07-22
1,2,Soup,22,2025-03-24,791,Grocery Store,West James,Non-Vegetarian,Dinner,2025-07-22
2,3,Fruits,46,2025-03-28,478,Catering Service,Lake Regina,Vegan,Breakfast,2025-07-22
3,4,Fruits,15,2025-03-16,930,Restaurant,Kellytown,Vegan,Lunch,2025-07-22
4,5,Soup,14,2025-03-19,279,Restaurant,Garciaport,Vegan,Dinner,2025-07-22
...,...,...,...,...,...,...,...,...,...,...
995,996,Fish,15,2025-03-30,467,Catering Service,Phillipsfort,Vegan,Breakfast,2025-07-22
996,997,Fish,22,2025-03-18,35,Grocery Store,Andersonmouth,Vegetarian,Breakfast,2025-07-22
997,998,Fruits,6,2025-03-22,444,Restaurant,New Billy,Non-Vegetarian,Dinner,2025-07-22
998,999,Pasta,15,2025-03-30,702,Supermarket,Lake Mistyton,Non-Vegetarian,Lunch,2025-07-22


8.Query to find Number of meals by meal type.

In [9]:
query = "SELECT meal_type, COUNT(*) FROM food_listings GROUP BY meal_type;"
df = pd.read_sql(query, get_connection())
print("8. Number of meals by meal type")
df

8. Number of meals by meal type


C:\Users\JAI\AppData\Local\Temp\ipykernel_888\3602350126.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, get_connection())


,meal_type,count
0,Dinner,245
1,Breakfast,254
2,Lunch,248
3,Snacks,253


9.Query to find Top 5 cities with most food listings.

In [10]:
query = """
SELECT city, COUNT(*) AS total
FROM providers
JOIN food_listings ON providers.provider_id = food_listings.provider_id
GROUP BY city
ORDER BY total DESC
LIMIT 5;
"""
df = pd.read_sql(query, get_connection())
print("9. Top 5 cities")
df

9. Top 5 cities


C:\Users\JAI\AppData\Local\Temp\ipykernel_888\2862109146.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, get_connection())


,city,total
0,New Carol,6
1,South Kathryn,6
2,East Angela,5
3,Perezport,5
4,Jimmyberg,5


10.Query to find How many foods listed by each provider.

In [11]:
query = "SELECT provider_id, COUNT(*) AS total_listed FROM food_listings GROUP BY provider_id;"
df = pd.read_sql(query, get_connection())
print("10. Foods listed by each provider")
df

10. Foods listed by each provider


C:\Users\JAI\AppData\Local\Temp\ipykernel_888\222776887.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, get_connection())


,provider_id,total_listed
0,828,2
1,938,3
2,184,1
3,87,3
4,753,1
...,...,...
632,520,1
633,912,2
634,55,1
635,148,1


11.Query to find Total claims by each receiver.

In [12]:
query = "SELECT receiver_id, COUNT(*) AS total_claims FROM claims GROUP BY receiver_id;"
df = pd.read_sql(query, get_connection())
print("11. Claims by each receiver")
df

11. Claims by each receiver


C:\Users\JAI\AppData\Local\Temp\ipykernel_888\556161595.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, get_connection())


,receiver_id,total_claims
0,938,1
1,87,2
2,184,1
3,652,1
4,477,1
...,...,...
619,901,4
620,536,2
621,449,1
622,742,5


12.Query to find Food items that expired.

In [13]:
query = "SELECT * FROM food_listings WHERE expiry_date::date < CURRENT_DATE;"
df = pd.read_sql(query, get_connection())
print("12. Expired food items")
df

12. Expired food items


C:\Users\JAI\AppData\Local\Temp\ipykernel_888\3489250352.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, get_connection())


,food_id,food_name,quantity,expiry_date,provider_id,provider_type,location,food_type,meal_type,listed_date
0,1,Bread,43,2025-03-17,110,Grocery Store,South Kellyville,Non-Vegetarian,Breakfast,2025-07-22
1,2,Soup,22,2025-03-24,791,Grocery Store,West James,Non-Vegetarian,Dinner,2025-07-22
2,3,Fruits,46,2025-03-28,478,Catering Service,Lake Regina,Vegan,Breakfast,2025-07-22
3,4,Fruits,15,2025-03-16,930,Restaurant,Kellytown,Vegan,Lunch,2025-07-22
4,5,Soup,14,2025-03-19,279,Restaurant,Garciaport,Vegan,Dinner,2025-07-22
...,...,...,...,...,...,...,...,...,...,...
995,996,Fish,15,2025-03-30,467,Catering Service,Phillipsfort,Vegan,Breakfast,2025-07-22
996,997,Fish,22,2025-03-18,35,Grocery Store,Andersonmouth,Vegetarian,Breakfast,2025-07-22
997,998,Fruits,6,2025-03-22,444,Restaurant,New Billy,Non-Vegetarian,Dinner,2025-07-22
998,999,Pasta,15,2025-03-30,702,Supermarket,Lake Mistyton,Non-Vegetarian,Lunch,2025-07-22


13.Query To Get Food and the receiver who claimed it.

In [14]:
query = """
SELECT f.food_name, r.name AS receiver_name, c.status, c.timestamp
FROM claims c
JOIN food_listings f ON c.food_id = f.food_id
JOIN receivers r ON c.receiver_id = r.receiver_id;
"""
df = pd.read_sql(query, get_connection())
print("13. Claimed food and receivers")
df

13. Claimed food and receivers


C:\Users\JAI\AppData\Local\Temp\ipykernel_888\2745426179.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, get_connection())


,food_name,receiver_name,status,timestamp
0,Dairy,Abigail Crawford,Pending,2025-03-05 05:26:00
1,Fruits,Regina Munoz,Cancelled,2025-03-11 10:24:00
2,Salad,Julie Lewis,Completed,2025-03-21 00:59:00
3,Fruits,Jodi Lee,Cancelled,2025-03-04 09:08:00
4,Pasta,Tina Watkins,Pending,2025-03-14 15:17:00
...,...,...,...,...
995,Salad,Judy Griffith,Completed,2025-03-13 19:40:00
996,Chicken,Kenneth Barnes,Cancelled,2025-03-17 22:04:00
997,Fish,Samantha Sherman,Cancelled,2025-03-13 18:00:00
998,Soup,Alvin West,Completed,2025-03-01 15:31:00


14.Query To Find List of unclaimed food.

In [15]:
query = """
SELECT f.*
FROM food_listings f
LEFT JOIN claims c ON f.food_id = c.food_id
WHERE c.food_id IS NULL;
"""
df = pd.read_sql(query, get_connection())
print("14. Unclaimed food")
df


14. Unclaimed food


C:\Users\JAI\AppData\Local\Temp\ipykernel_888\572815765.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, get_connection())


,food_id,food_name,quantity,expiry_date,provider_id,provider_type,location,food_type,meal_type,listed_date
0,2,Soup,22,2025-03-24,791,Grocery Store,West James,Non-Vegetarian,Dinner,2025-07-22
1,4,Fruits,15,2025-03-16,930,Restaurant,Kellytown,Vegan,Lunch,2025-07-22
2,5,Soup,14,2025-03-19,279,Restaurant,Garciaport,Vegan,Dinner,2025-07-22
3,8,Rice,49,2025-03-28,343,Catering Service,South Christopherborough,Vegetarian,Lunch,2025-07-22
4,10,Rice,41,2025-03-20,528,Supermarket,Markport,Vegan,Dinner,2025-07-22
...,...,...,...,...,...,...,...,...,...,...
348,987,Dairy,6,2025-03-29,557,Restaurant,East Aaron,Vegetarian,Dinner,2025-07-22
349,993,Soup,8,2025-03-28,470,Restaurant,Port Daniellechester,Vegetarian,Lunch,2025-07-22
350,994,Salad,17,2025-03-29,37,Catering Service,New Amanda,Non-Vegetarian,Snacks,2025-07-22
351,996,Fish,15,2025-03-30,467,Catering Service,Phillipsfort,Vegan,Breakfast,2025-07-22


15.Query To Get Provider details with food items listed.

In [16]:
query = """
SELECT p.name, p.city, f.food_name, f.quantity, f.expiry_date
FROM providers p
JOIN food_listings f ON p.provider_id = f.provider_id;
"""
df = pd.read_sql(query, get_connection())
print("15. Provider details with food listed")
df

15. Provider details with food listed


C:\Users\JAI\AppData\Local\Temp\ipykernel_888\2496959284.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, get_connection())


,name,city,food_name,quantity,expiry_date
0,Figueroa-Soto,South Kellyville,Bread,43,2025-03-17
1,Aguilar Group,West James,Soup,22,2025-03-24
2,"Lopez, Roach And Roach",Lake Regina,Fruits,46,2025-03-28
3,Cannon-Garcia,Kellytown,Fruits,15,2025-03-16
4,"Greene, Wood And Hernandez",Garciaport,Soup,14,2025-03-19
...,...,...,...,...,...
995,"Wilson, Watkins And Moore",Phillipsfort,Fish,15,2025-03-30
996,Nguyen-Parsons,Andersonmouth,Fish,22,2025-03-18
997,"King, Schneider And Banks",New Billy,Fruits,6,2025-03-22
998,Moreno-Melton,Lake Mistyton,Pasta,15,2025-03-30


16.Query To Find Claimed vs Unclaimed Food.

In [17]:
query = """
SELECT
    (SELECT COUNT(*) FROM claims) AS total_claimed,
    (SELECT COUNT(*) FROM food_listings) - (SELECT COUNT(DISTINCT food_id) FROM claims) AS total_unclaimed;
"""
df = pd.read_sql(query, get_connection())
print("16. Claimed vs Unclaimed")
df

16. Claimed vs Unclaimed


C:\Users\JAI\AppData\Local\Temp\ipykernel_888\642135054.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, get_connection())


,total_claimed,total_unclaimed
0,1000,353


17.Query To Find Most claimed food item.

In [18]:
query = """
SELECT food_id, COUNT(*) AS total_claims
FROM claims
GROUP BY food_id
ORDER BY total_claims DESC
LIMIT 1;
"""
df = pd.read_sql(query, get_connection())
print("17. Most claimed food item")
df


17. Most claimed food item


C:\Users\JAI\AppData\Local\Temp\ipykernel_888\187953434.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, get_connection())


,food_id,total_claims
0,548,5


Creating a Streamlit App with Filters

Now, we'll build a Streamlit app to display the results of our SQL queries.

In [19]:
import streamlit as st
import psycopg2
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta

# --- PostgreSQL Connection ---
def get_connection():
    return psycopg2.connect(
        dbname="food_wastage_db",
        user="postgres",
        password="Kirthu7625", 
        host="localhost",
        port="5432"
    )

# --- Load entire table ---
def load_data(table):
    conn = get_connection()
    df = pd.read_sql(f"SELECT * FROM {table};", conn)
    conn.close()
    return df

# --- Streamlit APP Title ---
st.title("🥦 Food Wastage Management Dashboard")
st.markdown("""
## 📘 Project Introduction

This **Food Wastage Management System** is a smart platform designed to minimize food waste by efficiently connecting **food providers** (like restaurants, hotels, or homes) with **receivers** (such as NGOs, shelters, or individuals in need).

🔍 **Features**:
- View and manage food listings, providers, and claims.
- Run insightful SQL queries with visualizations.
- Track claimed vs unclaimed food items.
- Add, update, or delete records in real-time.
- View food availability by type, expiry, and location.
- Interactive filters for better data exploration.

🌱 Let's work together to ensure **excess food is never wasted but redirected** to those who need it!
""")
st.markdown("""
### 👩‍💻 Creator: **Anupriya R**
#### 🧠 Skills & Tools Used:
- 🐍 Python  
- 🌐 Streamlit  
- 🗃️ PostgreSQL  
- 🧾 SQL  
- 📊 Data Visualization (Plotly, Matplotlib)  
- 🧮 Pandas  
- 🔄 CRUD Operations  
""")

menu = st.sidebar.selectbox("Select Option", ["View Tables", "Run Queries", "Visualize Data", "Add Record", "Update Record", "Delete Record"])

# --- View Tables ---
if menu == "View Tables":
    table = st.selectbox("Choose Table", ["providers", "receivers", "food_listings", "claims"])
    df = load_data(table)
    st.dataframe(df)

# --- SQL Queries Output Section ---
elif menu == "Run Queries":
    st.subheader("📄 Run Predefined SQL Queries")
    query_options = {
        "Show all providers": "SELECT * FROM providers;",
        "Recent 10 claims (most recent first)": "SELECT * FROM claims ORDER BY timestamp DESC LIMIT 10;",
        "List food claimed status": "SELECT * FROM claims WHERE status = 'Claimed';",
        "Foods listed in current month": """
            SELECT * FROM food_listings
            WHERE DATE_TRUNC('month', listed_date) = DATE_TRUNC('month', CURRENT_DATE);
        """,
        "List of unclaimed food": """
            SELECT f.*
            FROM food_listings f
            LEFT JOIN claims c ON f.food_id = c.food_id
            WHERE c.food_id IS NULL;
        """,
        "Food items that expired": "SELECT * FROM food_listings WHERE expiry_date::date < CURRENT_DATE;",
        "Food and the receiver who claimed it": """
            SELECT f.food_name, r.name AS receiver_name, c.status, c.timestamp
            FROM claims c
            JOIN food_listings f ON c.food_id = f.food_id
            JOIN receivers r ON c.receiver_id = r.receiver_id;
        """,
        "Provider details with food items listed": """
            SELECT p.name, p.city, f.food_name, f.quantity, f.expiry_date
            FROM providers p
            JOIN food_listings f ON p.provider_id = f.provider_id;
        """
    }

    selected_query = st.selectbox("Choose a Query", list(query_options.keys()))
    query = query_options[selected_query]

    try:
        conn = get_connection()
        df = pd.read_sql(query, conn)
        conn.close()
        st.dataframe(df)
    except Exception as e:
        st.error(f"Error: {e}")

# --- Data Visualizations + Filters ---
elif menu == "Visualize Data":
    st.subheader("📊 Data Visualizations with Filters")
    viz_options = [
        "Claims per Day (line chart)",
        "Food Listings by Type (bar chart)",
        "Meal Type Distribution (pie chart)",
        "Top 5 Cities by Food Listings",
        "Claimed vs Unclaimed Food (pie chart)",
        "Most Claimed Food Item"
    ]
    selected_viz = st.selectbox("Choose Visualization", viz_options)

    conn = get_connection()

    if selected_viz == "Claims per Day (line chart)":
        start_date = st.date_input("Start Date", datetime.now() - timedelta(days=30))
        end_date = st.date_input("End Date", datetime.now())

        df = pd.read_sql(f"""
            SELECT DATE(timestamp) AS claim_date, COUNT(*) AS total_claims
            FROM claims
            WHERE DATE(timestamp) BETWEEN '{start_date}' AND '{end_date}'
            GROUP BY claim_date
            ORDER BY claim_date;
        """, conn)
        st.dataframe(df)
        fig = px.line(df, x='claim_date', y='total_claims', title='Claims Per Day')
        st.plotly_chart(fig)

    elif selected_viz == "Food Listings by Type (bar chart)":
        df = pd.read_sql("SELECT food_type, COUNT(*) AS total FROM food_listings GROUP BY food_type;", conn)
        st.dataframe(df)
        fig = px.bar(df, x='food_type', y='total', title='Food Listings by Type', color='food_type')
        st.plotly_chart(fig)

    elif selected_viz == "Meal Type Distribution (pie chart)":
        df = pd.read_sql("SELECT meal_type, COUNT(*) AS total FROM food_listings GROUP BY meal_type;", conn)
        st.dataframe(df)
        fig = px.pie(df, names='meal_type', values='total', title='Meal Type Distribution')
        st.plotly_chart(fig)

    elif selected_viz == "Top 5 Cities by Food Listings":
        df = pd.read_sql("""
            SELECT city, COUNT(*) AS total
            FROM providers
            JOIN food_listings ON providers.provider_id = food_listings.provider_id
            GROUP BY city
            ORDER BY total DESC
            LIMIT 5;
        """, conn)
        st.dataframe(df)
        fig = px.bar(df, x='city', y='total', title='Top 5 Cities by Listings', color='city')
        st.plotly_chart(fig)

    elif selected_viz == "Claimed vs Unclaimed Food (pie chart)":
        claimed = pd.read_sql("SELECT COUNT(*) AS total FROM claims;", conn).iloc[0]['total']
        total_food = pd.read_sql("SELECT COUNT(*) AS total FROM food_listings;", conn).iloc[0]['total']
        unclaimed = total_food - pd.read_sql("SELECT COUNT(DISTINCT food_id) AS total FROM claims;", conn).iloc[0]['total']
        df = pd.DataFrame({'Status': ['Claimed', 'Unclaimed'], 'Count': [claimed, unclaimed]})
        fig = px.pie(df, names='Status', values='Count', title='Claimed vs Unclaimed Food')
        st.plotly_chart(fig)

    elif selected_viz == "Most Claimed Food Item":
        df = pd.read_sql("""
            SELECT f.food_name, COUNT(*) AS total_claims
            FROM claims c
            JOIN food_listings f ON c.food_id = f.food_id
            GROUP BY f.food_name
            ORDER BY total_claims DESC
            LIMIT 1;
        """, conn)
        st.dataframe(df)
        fig = px.bar(df, x='food_name', y='total_claims', title='Most Claimed Food Item')
        st.plotly_chart(fig)

    conn.close()

# --- Add, Update, Delete Record logic placeholders (same as before) ---
elif menu == "Add Record":
    st.subheader("➕ Add New Record")
    table = st.selectbox("Select Table to Insert", ["providers", "receivers", "food_listings", "claims"])
    st.info(f"Data entry form for `{table}` coming soon!")

elif menu == "Update Record":
    st.subheader("✏️ Update Existing Record")
    st.info("Update form for existing records coming soon!")

elif menu == "Delete Record":
    st.subheader("🗑️ Delete Record")
    st.info("Delete by ID feature coming soon!")







2025-07-23 00:27:49.333 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 00:27:52.263 
  command:

    streamlit run C:\Users\JAI\AppData\Local\Programs\Python\Python313\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-07-23 00:27:52.264 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 00:27:52.265 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 00:27:52.267 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 00:27:52.269 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 00:27:52.270 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-23 00:27:52

In [ ]:
!streamlit run app.py